In [1]:
! pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=dc934dc3444e082f79cff992fd4fd8266349127727ed493c3488ecf843a390f5
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
# Importing necessary Dependencies/Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark import SparkFiles , SparkContext
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sklearn.linear_model import LinearRegression , LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split


In [4]:
df = pd.read_csv("All.csv")

df.head()

,sector,Name,symbol,code,year,Current ratio,Quick Ratio,Cash ratio,Operating cash flow ratio,Working capital ratio,...,EV / Free Cash Flow,EV / Invested Capital,EV / Revenue,P/E Ratio,Price/Book,Dividend Per Share,Altman Z-Score,Piotroski Score,Classification,ML Classification
0,Academic & Education,Global Education Communities,GEC,A&E 01,2013,0.63,0.61,0.20,-0.10,0.63,...,-4.0x,0.9x,0.9x,-5.9x,1.1x,0.0,-3.1,0,0,NaN
1,Academic & Education,Global Education Communities,GEC,A&E 01,2014,0.78,0.76,0.16,-0.25,0.78,...,-2.6x,0.7x,0.5x,-14.3x,1.0x,0.0,-0.9,6,0,NaN
2,Academic & Education,Global Education Communities,GEC,A&E 01,2015,0.27,0.26,0.05,0.08,0.27,...,16.6x,1.0x,1.9x,-18.7x,1.1x,0.0,-0.7,4,0,NaN
3,Academic & Education,Global Education Communities,GEC,A&E 01,2016,0.76,0.73,0.27,-0.04,0.76,...,84.6x,1.1x,2.4x,6.4x,1.5x,0.0,2.9,4,1,NaN
4,Academic & Education,Global Education Communities,GEC,A&E 01,2017,0.32,0.31,0.12,0.18,0.32,...,28.4x,1.2x,3.4x,30.6x,1.8x,0.0,1.9,4,1,NaN


In [5]:
# Replacing all the null values with 0
df.fillna(0, inplace=True)

# Replacing x suffix from the DataFrame

columns_to_clean = ['EV/EBITDA', 'EV / EBIT', 'EV / Free Cash Flow',
                    'EV / Invested Capital', 'EV / Revenue', 'P/E Ratio', 'Price/Book']
df[columns_to_clean] = df[columns_to_clean].applymap(lambda x: x.rstrip('x') if isinstance(x, str) else x)

df.head()


,sector,Name,symbol,code,year,Current ratio,Quick Ratio,Cash ratio,Operating cash flow ratio,Working capital ratio,...,EV / Free Cash Flow,EV / Invested Capital,EV / Revenue,P/E Ratio,Price/Book,Dividend Per Share,Altman Z-Score,Piotroski Score,Classification,ML Classification
0,Academic & Education,Global Education Communities,GEC,A&E 01,2013,0.63,0.61,0.20,-0.10,0.63,...,-4.0,0.9,0.9,-5.9,1.1,0.0,-3.1,0,0,0.0
1,Academic & Education,Global Education Communities,GEC,A&E 01,2014,0.78,0.76,0.16,-0.25,0.78,...,-2.6,0.7,0.5,-14.3,1.0,0.0,-0.9,6,0,0.0
2,Academic & Education,Global Education Communities,GEC,A&E 01,2015,0.27,0.26,0.05,0.08,0.27,...,16.6,1.0,1.9,-18.7,1.1,0.0,-0.7,4,0,0.0
3,Academic & Education,Global Education Communities,GEC,A&E 01,2016,0.76,0.73,0.27,-0.04,0.76,...,84.6,1.1,2.4,6.4,1.5,0.0,2.9,4,1,0.0
4,Academic & Education,Global Education Communities,GEC,A&E 01,2017,0.32,0.31,0.12,0.18,0.32,...,28.4,1.2,3.4,30.6,1.8,0.0,1.9,4,1,0.0


In [6]:
df.dtypes

sector                                object
Name                                  object
symbol                                object
code                                  object
year                                   int64
Current ratio                        float64
Quick Ratio                          float64
Cash ratio                           float64
Operating cash flow ratio            float64
Working capital ratio                float64
Return On Equity (ROE)               float64
Net profit margin                    float64
Sales turnover ratio                 float64
Return On Assets (ROA)               float64
EPS                                  float64
Debt-to-assets ratio                 float64
Equity Ratio                         float64
Debt to equity ratio                 float64
Interest coverage ratio               object
The Long-Term Debt Coverage Ratio     object
Total debt to EBITDA ratio            object
Cash flow to debt ratio              float64
Inventory 

In [9]:

# List of columns to convert to float
columns_to_convert = [
    "Current ratio",
    "Quick Ratio",
    "Cash ratio",
    "Operating cash flow ratio",
    "Working capital ratio",
    "Return On Equity (ROE)",
    "Net profit margin",
    "Sales turnover ratio",
    "Return On Assets (ROA)",
    "Debt-to-assets ratio",
    "Equity Ratio",
    "Debt to equity ratio",
    "Interest coverage ratio",
    "The Long-Term Debt Coverage Ratio",
    "Total debt to EBITDA ratio",
    "Cash flow to debt ratio",
    "Inventory turnover ratio",
    "Receivables turnover ratio",
    "Asset Turnover Ratio",
    "Working capital turnover ratio",
    "EBITDA",
    "EV/EBITDA",
    "EV / EBIT",
    "EV / Free Cash Flow",
    "EV / Invested Capital",
    "EV / Revenue",
    "P/E Ratio",
    "Price/Book"
]

# Remove commas from values and convert to float
for column in columns_to_convert:
    df[column] = df[column].replace(',', '', regex=True).astype(float)


In [10]:
df.dtypes

sector                                object
Name                                  object
symbol                                object
code                                  object
year                                   int64
Current ratio                        float64
Quick Ratio                          float64
Cash ratio                           float64
Operating cash flow ratio            float64
Working capital ratio                float64
Return On Equity (ROE)               float64
Net profit margin                    float64
Sales turnover ratio                 float64
Return On Assets (ROA)               float64
EPS                                  float64
Debt-to-assets ratio                 float64
Equity Ratio                         float64
Debt to equity ratio                 float64
Interest coverage ratio              float64
The Long-Term Debt Coverage Ratio    float64
Total debt to EBITDA ratio           float64
Cash flow to debt ratio              float64
Inventory 

In [20]:
df.describe()

,year,Current ratio,Quick Ratio,Cash ratio,Operating cash flow ratio,Working capital ratio,Return On Equity (ROE),Net profit margin,Sales turnover ratio,Return On Assets (ROA),...,EV / Free Cash Flow,EV / Invested Capital,EV / Revenue,P/E Ratio,Price/Book,Dividend Per Share,Altman Z-Score,Piotroski Score,Classification,ML Classification
count,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,...,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.0
mean,2017.645705,3.904716,2.653565,2.362588,1.137754,3.904716,0.205299,-4.977635,0.572574,-1.859714,...,15.988883,1.925154,86.081022,33.268669,7.143908,0.634243,7.667266,5.170129,1.781022,0.0
std,2.853880,13.281136,20.833486,12.003700,4.793677,13.281136,6.486306,154.571110,0.740058,77.349164,...,795.340966,5.182364,2492.683588,561.236621,169.321484,0.931089,19.439648,1.836804,0.584730,0.0
min,2013.000000,0.000000,-723.450000,0.000000,-52.770000,0.000000,-22.909100,-4755.043900,-0.165700,-3264.000000,...,-19648.700000,-164.500000,-88.000000,-2095.400000,-1221.600000,0.000000,-332.800000,0.000000,0.000000,0.0
25%,2015.000000,0.910000,0.660000,0.110000,0.280000,0.910000,-0.009400,0.000000,0.174900,-0.006200,...,-5.700000,1.000000,1.100000,0.000000,1.100000,0.000000,4.200000,4.000000,2.000000,0.0
50%,2018.000000,1.430000,1.050000,0.360000,0.640000,1.430000,0.058200,0.055800,0.334700,0.030600,...,14.900000,1.400000,2.900000,15.800000,1.900000,0.294000,5.600000,5.000000,2.000000,0.0
75%,2020.000000,2.670000,1.820000,1.010000,1.420000,2.670000,0.141400,0.145200,0.819300,0.062900,...,29.100000,2.200000,5.200000,29.700000,3.300000,0.936000,7.900000,7.000000,2.000000,0.0
max,2022.000000,306.700000,190.880000,306.190000,102.000000,306.700000,267.994700,344.819200,20.079500,6.197700,...,18704.700000,50.800000,93462.700000,22852.500000,6880.000000,13.680000,378.800000,9.000000,2.000000,0.0


In [17]:
render_username = 'forecating_companies_future_user'
render_password = 'qRvkrYzSYuo6TvDWAh8SMcQokhT5pYyb'
render_host = 'dpg-cog4tdmv3ddc73e67q00-a.ohio-postgres.render.com'
database = 'forecating_companies_future'
engine = create_engine(f"postgresql+psycopg2://{render_username}:{render_password}@{render_host}:5432/{database}")

In [19]:
df.to_sql('Company_Data', engine)

147